In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from collections import defaultdict
import os, sys
sys.path.insert(0, "/mnt/f/dev/git/miRExplore/python/")


import scispacy
import spacy
from spacy import displacy

#nlp = spacy.load('/mnt/d/spacy/models/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')




In [ ]:
nlp_weblg = spacy.load('/mnt/d/spacy/models/en_core_web_lg-2.2.0/en_core_web_lg/en_core_web_lg-2.2.0/')

In [3]:
nlp = spacy.load('/mnt/f/spacy/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')
nlp_ent = spacy.load("/mnt/f/spacy/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")


In [ ]:
nlp = spacy.load('/mnt/f/spacy/en_core_sci_lg-0.2.5/en_core_sci_lg/en_core_sci_lg-0.2.5/')
nlp_ent = spacy.load("/mnt/f/spacy/en_ner_bionlp13cg_md-0.2.5/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.5")

In [ ]:
!ls /mnt/d/spacy/models/en_core_web_lg-2.2.0/en_core_web_lg/en_core_web_lg-2.2.0

In [ ]:
import textmining.MirGeneRelCheck


In [ ]:

def analyse(testCase, dist=80):
    relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

    doc = nlp(testCase[0])
    
    lWord = doc[testCase[1]]
    rWord = doc[testCase[2]]

    for t in doc:
        print(t.idx, t.i, t, t.pos_, t.dep_, t.head, t.head.idx)

    print(lWord, rWord)

    if not "mir" in str(lWord).lower() and not "let" in str(lWord).lower():
        tmp = lWord
        lWord = rWord
        rWord = tmp
        
    print(lWord, rWord)
    
    checkResults = relCheck.checkRelation(doc, lWord, rWord, verbose=True)
    print(checkResults)
    
    displacy.render(doc, style="dep", options={"compact": True, "distance": dist})

In [ ]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Functional association between miR 124a and CDK6 in medulloblastoma was established using luciferase assays.", 4, 6)

analyse(testcase)

In [ ]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Overexpression of mir-7 and mir-153 significantly reduces endogenous alpha-synuclein levels, whereas inhibition of mir-7 and mir-153 enhances translation of a luciferase construct bearing the alpha-s", 6, 10)

analyse(testcase)

In [ ]:
doc = nlp(testcase[0])

allTokens = {t.i: t for t in doc}

In [ ]:
[x for x in allTokens[23].children]

In [ ]:
allRels[:5]

In [ ]:
%aimport textmining.MirGeneRelCheck

testCase = (u"We show here that miR-29a, -29b-1, and -9 can regulate BACE1 expression in vitro.", 4, 12)
testCase = (u"It has been predicted that miR-146a may target Tata Binding Protein (TBP).",5, 12)
testCase = (u"Our results showed that the expression of miR-106b was inversely correlated with TβRII protein levels and miR-106b can directly inhibit the TβRII translation in vitro.", 7, 21)

relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

doc = nlp(testCase[0])

mirword = doc[testCase[1]]
geneword = doc[testCase[2]]

verbose=False

In [ ]:
conjResult, conjs = relCheck.checkCommonConj(doc, mirword, geneword, verbose)
sdpPass, passive, negated = relCheck.checkSDP(doc, mirword, geneword, verbose)
compPass = relCheck.checkCompartments(doc, mirword, geneword, verbose)
sigPathway = relCheck.checkSurContext(doc, mirword, geneword, verbose)


In [ ]:
sdpRes = relCheck._get_sdp_path(doc, mirword.i, geneword.i)
sdpRes

In [ ]:
%aimport textmining.MirGeneRelCheck

testCase = (u"We show here that miR-29a, -29b-1, and -9 can regulate BACE1 expression in vitro.", 4, 12)
testCase = (u"It has been predicted that miR-146a may target Tata Binding Protein (TBP).",5, 12)
testCase = (u"Our results showed that the expression of miR-106b was inversely correlated with TβRII protein levels and miR-106b can directly inhibit the TβRII translation in vitro.", 7, 21)
testCase = (u"The reduction of the affinity of miR-433 to the 3' UTR would result in increased FGF20 expression and upregulation of alpha-synuclein, which could in turn promote dopaminergic neurons degeneration.", 6, 16)
testCase = (u"These findings identify miR-29b as a novel posttranscriptional regulator of PGRN expression, raising the possibility that miR-29b or other miRNAs might be targeted therapeutically to increase hPGRN levels in some FTD patients.", 3,10)
relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

doc = nlp(testCase[0])

mirword = doc[testCase[1]]
geneword = doc[testCase[2]]

verbose=False

from collections import Counter



In [ ]:
def loadStems():
    
    print("Loading stems")
    allRels = []
    stem2class = {}

    renameClass = {
        'INT': 'NEU',
        "BOOST": "POS",
        "COMBINE": "NEU",
        "CHANGE": "NEU",
    }

    with open("/mnt/d/dev/git/miRExplore/python/allrels.csv") as fin:

        for line in fin:
            aline = line.strip().split(",", 1)
            aregSyns = aline[1].split(":")
            regSyns = []
            if len(aregSyns) > 1:
                regSyns = aregSyns[1].split("|")
            
            aline[1] = aline[1].split(":")[0]           

            regClass = aline[0]
            regStem = aline[1]

            if regClass in renameClass:
                regClass = renameClass[regClass]
                
            stem2class[regStem] = regClass
                
            for regSyn in regSyns:
                if not regSyn in stem2class:
                    stem2class[regSyn] = regClass


            allRels.append((regClass, regStem))

    allStems = [x for x in stem2class]
    
    return allStems, allRels, stem2class

allStems, allRels, stem2class = loadStems()


In [4]:
from textmining.MirGeneRelCheck import SentenceRelationChecker, SentenceRelationClassifier
from utils.tmutils import normalize_gene_names
from collections import Counter
from lxml import etree
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from itertools import chain, combinations


In [ ]:
scaiBase = "/mnt/d/owncloud/data/miRExplore/scai_corpus/"

base = "TRAIN"

if base == "TRAIN":
    scaiFile = "miRNA_train_fixed.xml"
    relsFile = "scai_train_rels.tsv"
elif base == "TEST":
    scaiFile = "miRNA_test_fixed.xml"
    relsFile = "scai_test_rels.tsv"
    
normGeneSymbols= normalize_gene_names(path=scaiBase + "/../obodir/" + "/hgnc_no_withdrawn.syn")

relChecker = SentenceRelationChecker(nlp)

correctIdentified = 0
incorrectIdentified = 0
totalChecks = 0
incorrectClass = Counter()


In [ ]:
relClasses = loadRelations()

In [ ]:
nlp = spacy.load('/mnt/f/spacy/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')
nlp_ent = spacy.load("/mnt/f/spacy/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")



In [5]:
%aimport textmining.MirGeneRelCheck

scaiBase = "/mnt/d/owncloud/data/miRExplore/scai_corpus/"

normGeneSymbols= normalize_gene_names(path=scaiBase + "/../obodir/" + "/hgnc_no_withdrawn.syn")

relChecker = SentenceRelationChecker(nlp, nlp_ent=nlp_ent)
relClassifier = SentenceRelationClassifier()


def loadRelations(relsFile):
    reldir2new = {}
    reldir2new["TARGET"] = "NEU"
    reldir2new["INVCORR"] = "NEU"
    reldir2new["REGULATE"] = "NEU"
    relClasses = {}
    with open(os.path.join(scaiBase, relsFile)) as fin:
        
        print("Loading relations", relsFile)
        
        for lidx, line in enumerate(fin):

            if lidx == 0:
                continue

            line = line.strip().split("\t")

            interactionID = line[0].strip()
            relType = line[1].strip()
            relRDir = line[2].strip()
            relIndirect = line[3].strip()
            relPassive = line[4].strip()

            if relRDir in reldir2new:
                relRDir = reldir2new[relRDir]

            relClasses[interactionID] = (relType, relRDir, relIndirect, relPassive)
            
        print("Interactions", len(relClasses))
        print("Non NA Interactions", len([x for x in relClasses if not relClasses[x][0] == "NA"]))
            
    return relClasses

def all_subsets(ss):
    
    return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

def runCheck(numelems, base):
    
    if base == "TRAIN":
        scaiFile = "miRNA_train_fixed.xml"
        relsFile = "scai_train_rels.tsv"
    elif base == "TEST":
        scaiFile = "miRNA_test_fixed.xml"
        relsFile = "scai_test_rels.tsv"
    
    relClasses = loadRelations(relsFile)
    
    check2results = {}
    checkSubsets = [x for x in all_subsets(relClassifier.major_checks)]
    checkSubsets = [checkSubsets[-1]]
    
    print("Will test {} subsets.".format(len(checkSubsets)))
    
    for subset_checks in checkSubsets:
    
        print("Testing checks", subset_checks)
        
        #relClassifier.active_checks = subset_checks

        #allStems, allRels, stem2class = loadStems()

        with open(os.path.join(scaiBase, scaiFile), 'r') as fin:
            tree = etree.parse(fin)
            root = tree.getroot()
            scaiPairs = []
            totalChecks = 0
            correctIdentified = 0
            incorrectIdentified = 0

            incorrectClassified = 0
            correctClassified = 0
            totalClassified = 0
            correctInteractClassified = 0
            totalValidClassified = 0
            totalClassifiable = 0
            totalMirGeneDownClassified = 0

            errorByDetect = Counter()
            classifiedByDetect = Counter()
            elemCaseCounter = Counter()
            incorrectClass = Counter()

            classifyTrue = []
            classifyPred = []

            for elem in root.findall(".//document"):

                pmid = elem.attrib['origId']

                for sentElem in elem:

                    allEntities = sentElem.findall(".//entity")
                    allPairs = sentElem.findall(".//pair")

                    sentText = sentElem.attrib["text"]

                    entId2elem = {}

                    for entity in allEntities:
                        entId = entity.attrib['id']
                        entText = entity.attrib['text']
                        entType = entity.attrib['type']
                        entOffset = tuple([int(x) for x in entity.attrib['charOffset'].split("-")])

                        if entType in ["Specific_miRNAs", "Genes/Proteins"]:

                            if "Genes" in entType:
                                if entText in normGeneSymbols:
                                    entText = normGeneSymbols[entText]
                                elif entText.upper() in normGeneSymbols:
                                    gene = normGeneSymbols[entText.upper()]
                            else:
                                try:
                                    entText = miRNA(entText).getStringFromParts([miRNAPART.MATURE, miRNAPART.ID, miRNAPART.PRECURSOR])
                                except:
                                    pass

                            entTuple = (entText, entType, (entOffset[0], entOffset[1]+1))
                            entId2elem[entId] = entTuple


                            sentEntText = sentText[entTuple[2][0]:entTuple[2][1]]

                    for pair in allPairs:

                        validInteraction = pair.attrib['interaction'].lower() == "true"
                        pairE1 = pair.attrib['e1']
                        pairE2 = pair.attrib['e2']

                        #if pairInt == 'true':
                        if pairE1 in entId2elem and pairE2 in entId2elem:

                            totalChecks += 1

                            e1 = entId2elem[pairE1]
                            e2 = entId2elem[pairE2]

                            if not e1[1] in ["Specific_miRNAs"]:

                                tmp=e1
                                e1=e2
                                e2=tmp


                            relRes = relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=False
                                                                )


                            fullsentence = relRes['full_sentence']
                            acceptInteraction = relRes['accept_relation']

                            if not acceptInteraction == validInteraction:

                                incorrectIdentified += 1
                                
                                relResV=relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=True
                                                                )
                                
                                print(relResV)
                            else:
                                correctIdentified += 1

                            elemCaseCounter[(validInteraction, acceptInteraction)]+=1

                            totalClassified += 1

                            if validInteraction:# and acceptInteraction:
                                #print(relRes["check_results"])   
                                totalValidClassified += 1

                                foundClasses = relRes["check_results"]["classification"] #{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'regulation_keys': set()}
                                foundTuple = (foundClasses["interaction_dir"], foundClasses["regulation_dir"])                                
                                relationID = pair.attrib["id"]
                                testTuple = relClasses[relationID]

                                classifyTrue.append((testTuple[0], testTuple[1]))
                                classifyPred.append((foundTuple[0], foundTuple[1]))

                                if testTuple[0] == "MIR_GENE" and testTuple[1] == "DOWN":
                                    totalMirGeneDownClassified += 1

                                classifiedByDetect[foundClasses["reg_detect"]] += 1

                                if testTuple[0] != foundTuple[0] or testTuple[1] != foundTuple[1]:


                                    #print(sentText)
                                    #print(e1, e2)
                                    #print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                    #print(testTuple)

                                    #print()
                                    #print()
                                    #print()

                                    errorByDetect[foundClasses["reg_detect"]] += 1

                                    incorrectClassified += 1



                                    #numelems -= 1
                                    #if numelems == 0:
                                    #    return
                                else:
                                    
                                    if foundClasses["reg_detect_major"] == "return":
                                        print(foundClasses)
                                        print(sentText)
                                        print(e1, e2)
                                        print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                        print(testTuple)

                                        print()
                                        print()
                                        print()
                                    correctClassified += 1

                                    if acceptInteraction:
                                        correctInteractClassified += 1



            def printStats(outfile):
                print("Total:     ", totalChecks, file=outfile)
                print("Correct:   ", correctIdentified, correctIdentified/totalChecks, file=outfile)
                print("Incorrect: ", incorrectIdentified, incorrectIdentified/totalChecks, file=outfile)
                print("classes", incorrectClass, file=outfile)
                
                precision = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(True, False)])
                recall = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(False, True)])

                f1 = 2* precision * recall / (precision+recall)

                specificity = elemCaseCounter[(False, False)] / (elemCaseCounter[(True, False)] + elemCaseCounter[(False, False)])

                print("precision", precision, file=outfile)
                print("recall", recall, file=outfile)
                print("specificity", specificity, file=outfile)
                print("f1", f1, file=outfile)

                print("Correct classified:     ", correctClassified, correctClassified/totalClassified, correctClassified/totalValidClassified,file=outfile)
                print("Incorrect classified:   ", incorrectClassified, incorrectClassified/totalClassified, incorrectClassified/totalValidClassified,file=outfile)
                print("Random classified: ", totalMirGeneDownClassified, totalMirGeneDownClassified / totalClassified, totalMirGeneDownClassified/totalValidClassified, file=outfile)
                print(errorByDetect)

                print("Correct interaction&classified:     ", correctInteractClassified, correctInteractClassified/totalClassified, correctInteractClassified/totalValidClassified,file=outfile)

                print("Classified by: ", classifiedByDetect)

                lClassifyTrue = ["_".join(x) for x in classifyTrue]
                lClassifyPred = ["_".join(x) for x in classifyPred]

                allLabels = sorted(set(lClassifyTrue+lClassifyPred))
                matrix = multilabel_confusion_matrix(lClassifyTrue, lClassifyPred, labels=allLabels)
                print(matrix)
                print(classification_report(lClassifyTrue,lClassifyPred))
                
                return classification_report(lClassifyTrue, lClassifyPred, output_dict=True)

            rep = printStats(sys.stdout)
            check2results[subset_checks] = rep
        #printStats(sys.stderr)
    return check2results
        


In [7]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier()

test_results = runCheck(-1, "TEST")

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 138
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')
{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'return', 'reg_detect_major': 'return'}
Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the proliferation of CRL-1690 and CRL-2610 cell lines. 
('miRNA-128', 'Specific_miRNAs', (22, 31)) ('APOBEC3C', 'Genes/Proteins', (99, 103))
('MIR_GENE', 'NEU') return passive? False
('MIR_GENE', 'NEU', 'FALSE', 'FALSE')



{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'return', 'reg_detect_major': 'return'}
Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the p

In [ ]:
analyse(
#("We also found that miR-326 promoted T(H)-17 differentiation by targeting Ets-1, a negative regulator of T(H)-17 differentiation.", 4, 10)
    #("In CGNPs, the Shh effector N-myc, but not Gli1, induced miR-17/92 expression.", 4, 10),
    #("miR-23a directly bound the 3' UTR of XIAP, and miR-23a inhibition led to an increase in XIAP mRNA in vitro, demonstrating that XIAP is a previously uncharacterized target for miR-23a.", 11, 18),
    #("Downregulation of Pdcd4 by mir-21 facilitates glioblastoma proliferation in vivo.", 4,2),
    ("We show that SRF and myocardin regulate a cardiovascular-specific microRNA (miRNA) cluster encoding miR-143 and miR-145. ", 3, 15),
    100
)

In [ ]:
import pickle

with open("scai_test_f1.pickle", "wb") as fout:
    pickle.dump(test_results, fout)

In [ ]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier()

runCheck(-1, "TRAIN")

In [ ]:

def getTokenDist(tk1, tk2):
    return abs(tk1.i-tk2.i)

def countStems(tokens, geneword, mirword, keepNEU=False):
    stemTypes2Count = Counter()
    mirgene2stem = defaultdict(lambda: Counter())

    for tidx,x in enumerate(tokens):
        tknStem, matchStem, wasFound = getstem(x)

        if wasFound and (tknStem != "NEU" or keepNEU):
            
            if tidx >= len(tokens)-2 and str(tokens[tidx-1]) == "to":
                continue
            
            geneDist = getTokenDist(x, geneword)
            mirDist = getTokenDist(x, mirword)
                        
            if geneDist < mirDist and geneDist < 3:
                mirgene2stem["gene"][tknStem] += 1
            elif mirDist < geneDist and mirDist < 3:
                mirgene2stem["mir"][tknStem] += 1

        if wasFound and (tknStem != "NEU" or keepNEU):
            stemTypes2Count[tknStem] += 1

    if len(stemTypes2Count) == 0:
        if "regulated" in str(geneword):
            stemTypes2Count["NEU"] += 1

    return stemTypes2Count,mirgene2stem

def getRelsForTokens(tokens, desc,verbose):
    rels = []
    verbSet = set(["VERB"])
    for x in tokens:
        
        tokenStemClass, s, _ = getstem(x)
        rels.append((s, None, tokenStemClass, x.pos_, x))
        if verbose:
            print(desc, x, s, tokenStemClass, x.pos_, x)
                    
    hasVerb = len(list(set([x[3] for x in rels]) & verbSet)) > 0
    
    if hasVerb:
        rels = [x for x in rels if x[3] in verbSet]
    
    return rels

def evalCounts(dirReg, mirPos, mirNeg, genePos, geneNeg):

    invcor = False
    regdir = None

    #if dirReg == 0:
    if (mirPos > 0 and geneNeg > 0) or (genePos > 0 and mirNeg > 0):
        return "DOWN", True

    #else:
    if ((mirPos > 0 and genePos > 0) and(mirNeg == 0 and geneNeg == 0)) or ((mirNeg > 0 and geneNeg >0) and(mirPos == 0 and genePos == 0)):
        return "UP", False

    return None, None

def scoreWord(tkn):
    
    jtkstem, s, _ = getstem(tkn)
                        
    if jtkstem == None:

        if jtk.startswith(("loss", "decreas", "low")):
            return 0,1

        elif jtk.startswith(("increas", "high", "posit")):
            return 1,0

    else:
        if jtkstem == "POS":
            return 1,0
        elif jtkstem == "NEG":
            return 0,1
        
    return 0,0


def getTextBetween(tks, lword, rword):
    tokenBetween = [x for x in tks if lword.i-3 < x.i < rword.i+3]
    
    lit = lword
    while lit.pos_ in ["nmod", "conj"]:
        if not lit in tokenBetween:
            tokenBetween.append(lit)
            
        lit = lit.head
        
    rit = rword
    while rit.pos_ in ["nmod", "conj"]:
        if not rit in tokenBetween:
            tokenBetween.append(rit)
            
        rit = rit.head


    tokenBetween = list(sorted(tokenBetween, key=lambda x: x.i))
    tokenBetweenStrict = list(sorted([x for x in tks if lword.i < x.i < rword.i], key=lambda x: x.i))
    textBetween = ' '.join(" ".join([str(x) for x in tokenBetween]).split())
    textAfter = ' '.join(" ".join([str(x) for x in [y for y in rword.doc if rword.i <= y.i < rword.i+5]]).split())
    textBefore = ' '.join(" ".join([str(x) for x in [y for y in rword.doc if lword.i-5 <= y.i < lword.i]]).split())

    return tokenBetween, tokenBetweenStrict, textBetween, textBefore, textAfter

dir2opp = {"MIR_GENE": "GENE_MIR", "GENE_MIR": "MIR_GENE"}
idir2opp = {"POS": "NEG", "NEG": "POS"}
rdir2opp = {"DOWN": "UP", "UP": "DOWN"}

def testRelClassifier2(doc, mirword, geneword, relchecker, verbose):
    isPassive = relchecker.checkPassive(doc, mirword, geneword, verbose=False)
    compartment = relchecker.getCompartment(doc, mirword, geneword, verbose=False)
    
    if verbose:
        print("passive?", isPassive)
        
        
    sdpRes = relchecker._get_sdp_path(doc, mirword.i, geneword.i)
        
    stemTypes2Count = Counter()
    
    for x in sdpRes:
        tknStem, matchStem, _ = getstem(x[0])
        
        if tknStem != "NEU":
            stemTypes2Count[tknStem] += 1
            
    print(stemTypes2Count)
        
    if len(stemTypes2Count) > 0:
        regStem = stemTypes2Count.most_common(1)[0][0]
        
        if regStem == "NEG":
            regStem = "DOWN"
        elif regStem == "POS":
            regStem = "UP"
            
        if stemTypes2Count["POS"] == stemTypes2Count["NEG"]:
            regStem = "DOWN"
            
        idir = "MIR_GENE"
        if isPassive and mirword.i < geneword.i:
            idir = "GENE_MIR"
            
        #return {"regulation_dir": regStem, "interaction_dir": idir, "passive": isPassive, "reg_detect": "sdp counts"}
        
    targetWords = ["targets of", "target of", "regulator of", "targeting", "direct regulator"]
        
    if compartment != None:
        tokenBetween = compartment
        textBetween = ' '.join(" ".join([str(x) for x in tokenBetween]).split())
        
        if geneword.i < mirword.i:
            
            if any([x in textBetween for x in ["positive correlation"]]):
                return {"regulation_dir": "UP", "interaction_dir": "GENE_MIR", "passive": isPassive, "reg_detect": "compartment gene mir pos corr"}
            
            if any([x in textBetween for x in ["inverse correlation", "inversely correlated", "inversely regulated"]]):
                return {"regulation_dir": "DOWN", "interaction_dir": "GENE_MIR", "passive": isPassive, "reg_detect": "compartment gene mir neg corr"}
            
            if any([x in textBetween for x in targetWords]):
                return {"regulation_dir": "NEU", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "compartment gene mir"}
            
        elif mirword.i < geneword.i:
            
            if any([x in textBetween for x in ["positive correlation"]]):
                return {"regulation_dir": "UP", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "compartment mir gene pos corr"}
            
            if any([x in textBetween for x in ["inverse correlation", "inversely correlated", "inversely regulated"]]):
                return {"regulation_dir": "DOWN", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "compartment mir gene neg corr"}
            
            if any([x in textBetween for x in targetWords]):
                return {"regulation_dir": "NEU", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "compartment mir gene"}
        


    if True:
        
        tokSrc = compartment if compartment != None else doc
        
        if geneword.i < mirword.i:

            tokenBetween, tokenBetweenStrict, textBetween, textBefore, textAfter = getTextBetween(tokSrc, geneword, mirword)

            print(textBetween)
            
            if any([x in textBetween for x in ["positive correlation"]]):
                return {"regulation_dir": "UP", "interaction_dir": "GENE_MIR", "passive": isPassive, "reg_detect": "between gene mir pos corr"}
            
            if any([x in textBetween for x in ["inverse correlation", "inversely correlated", "inversely regulated"]]):
                return {"regulation_dir": "DOWN", "interaction_dir": "GENE_MIR", "passive": isPassive, "reg_detect": "between gene mir neg corr"}
            
            if any([x in textBetween for x in ["negative regulation of"]]):
                return {"regulation_dir": "UP", "interaction_dir": "GENE_MIR", "passive": isPassive, "reg_detect": "between gene mir reg corr"}
      
            if any([x in textBefore for x in ["binding site", "binding sites", "binding to", "binding directly to"]]):
                return {"regulation_dir": "NEU", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between gene mir sites"}
            
            
    
            if any([x in textBetween for x in targetWords]) or any([x in textBetween for x in ["target", "targeted by", "binding efficiency", "binding site for", "recognizing"]]):
                return {"regulation_dir": "NEU", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between gene mir"}
            

            #stemTypes2Count = Counter()
            #for x in tokenBetween:
            #    tknStem, matchStem = getstem(x)
            #    if tknStem != "NEU":
            #        stemTypes2Count[tknStem] += 1
                    
            if False and len(stemTypes2Count) > 0:
                regStem = stemTypes2Count.most_common(1)[0][0]
                if regStem == "NEG":
                    regStem = "DOWN"
                elif regStem == "POS":
                    regStem = "UP"
                    
                if stemTypes2Count["POS"] == stemTypes2Count["NEG"]:
                    regStem = "DOWN"

                idir = "MIR_GENE"
                if isPassive and geneword.i < mirword.i:
                    idir = "GENE_MIR"

                return {"regulation_dir": regStem, "interaction_dir": idir, "passive": isPassive, "reg_detect": "counts between gene mir"}
            
        else:
            #mirword.i < geneword.i
            tokenBetween, tokenBetweenStrict, textBetween, textBefore, textAfter = getTextBetween(tokSrc, mirword, geneword)
                        
            print(textBetween)
            
            if any([x in textBetween for x in ["positive correlation"]]) and stemTypes2Count.get("NEG", 0) == 0:
                return {"regulation_dir": "UP", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between mir gene pos corr"}
            
            if any([x in textBetween for x in ["negative regulation of"]]):
                return {"regulation_dir": "UP", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between mir gene reg corr"}
            
            if any([x in textBetween for x in ["inverse correlation", "inversely correlated", "inversely regulated"]]):
                return {"regulation_dir": "DOWN", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between mir gene neg corr"}
            
            if any([x in textBetween for x in ["directly inhibits", "directly represses", "translational inhibition" ]]):
                return {"regulation_dir": "DOWN", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between mir gene"}
            
            if any([x in textBetween for x in targetWords + ["directly regulated", "its target gene", "binding site"]]):
                return {"regulation_dir": "NEU", "interaction_dir": "MIR_GENE", "passive": isPassive, "reg_detect": "between mir gene"}
            
            if any([x in textBetween for x in ["negatively regulated by", "negative regulation of"]]):
                return {"regulation_dir": "DOWN", "interaction_dir": "GENE_MIR", "passive": isPassive, "reg_detect": "between mir gene nrb"}
            
            
        if len(tokenBetween) > 0:
                       
            stemTypes2Count,mirgene2stem = countStems(tokenBetween, geneword, mirword)
                    
            if len(stemTypes2Count) > 0:
                regStem = stemTypes2Count.most_common(1)[0][0]
                
                if len(stemTypes2Count) >= 2 and len(mirgene2stem["gene"]) >0 and len(mirgene2stem["mir"])>0 and mirgene2stem["gene"].most_common(1)[0][0] == idir2opp[mirgene2stem["mir"].most_common(1)[0][0]]:
                    print(mirgene2stem)
                    if geneword.i < mirword.i:
                        if mirgene2stem["gene"].most_common(1)[0][0] == "POS":
                            regDir = "MIR_GENE"
                            regStem = "DOWN"
                        else:
                            regDir = "MIR_GENE"
                            regStem = "DOWN"
                    else:
                                                
                        if mirgene2stem["mir"].most_common(1)[0][0] == "NEG":
                            regDir = "MIR_GENE"
                            regStem = "DOWN"
                                                        
                        else:
                            regDir = "MIR_GENE"
                            regStem = "DOWN"
                            
                        if "after" in textBetween:
                            regDir = dir2opp[regDir]
                            
                    if isPassive:
                        regDir = dir2opp[regDir]
                        regStem = rdir2opp[regStem]
                        
                    return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "counts opp"}
                    
                    
                else:
                    stemTypes2Count,mirgene2stem = countStems(tokenBetween, geneword, mirword, keepNEU=True)
                    print("counts between 1", stemTypes2Count)
                    regStem = stemTypes2Count.most_common(1)[0][0]
                    
                    if stemTypes2Count["POS"] == stemTypes2Count["NEG"] == 1 and stemTypes2Count["NEU"] <= 3:
                        regDir = "MIR_GENE"
                        regStem = "DOWN"
                        
                        if any(x in textBetween for x in ["after"]):
                            regDir = dir2opp[regDir]
                        
                        return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "counts between equal"}
                    
                    if stemTypes2Count["POS"] == 2 and stemTypes2Count["NEG"] == 1 and stemTypes2Count["NEU"] <= 2:
                        regDir = "MIR_GENE"
                        regStem = "DOWN"
                        
                        return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "counts between alternating"}
                    
                    if regStem == "NEU":
                        if stemTypes2Count["POS"] > 0 and stemTypes2Count["POS"] > stemTypes2Count["NEG"]:
                            regStem = "POS"
                            
                        elif stemTypes2Count["NEG"] > 0 and stemTypes2Count["NEG"] > stemTypes2Count["POS"]:
                            regStem = "NEG"
                        
                    
                    #regStem = "NEU"

                    if mirword.i < geneword.i:
                        regDir = "MIR_GENE"
                    else:
                        regDir = "GENE_MIR"
                        
                    if mirword.i < geneword.i and "-stressed" in textBetween:
                        regDir = "GENE_MIR"
                        
                    if mirword.i < geneword.i and str(doc[geneword.i-1]) in ["by", "with"]:
                        regDir = "GENE_MIR"

                        stemTypes2Count,mirgene2stem = countStems(tokenBetweenStrict, geneword, mirword)

                        if len(stemTypes2Count) > 0:
                            regStem = stemTypes2Count.most_common(1)[0][0]
                        #else:
                        #    regStem = "NEU"
                        print("counts between 2", stemTypes2Count, regStem)
                            
                    elif geneword.i < mirword.i and any([str(doc[mirword.i-j]) in ["by", "with"] for j in [1,2]]):
                        regDir = "MIR_GENE"

                        stemTypes2Count,mirgene2stem = countStems(tokenBetweenStrict, geneword, mirword)
                        if len(stemTypes2Count) > 0:
                            regStem = stemTypes2Count.most_common(1)[0][0]
                        #else:
                        #    regStem = "NEU"
                        print("counts between 2", stemTypes2Count, regStem)
                            
                    elif isPassive and geneword.i < mirword.i:
                        regDir = dir2opp[regDir]

                    if stemTypes2Count["POS"] > 0 and stemTypes2Count["POS"] == stemTypes2Count["NEG"]:
                        regStem = "NEU"                       
                        
                    
                        #regStem = idir2opp[regStem]
                    
                    if geneword.i < mirword.i:
                        if str(geneword).endswith("-regulating"):
                            regStem = "NEU"
                            
                    if mirword.i < geneword.i and "after the addition" in textBetween:
                        regStem = "DOWN"
                        regDir = "GENE_MIR"
                            
                        
                    if regStem == "NEG":
                        regStem = "DOWN"
                    elif regStem == "POS":
                        regStem = "UP"

                        
                    
                    return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "counts between"}
    
        
    regDir = "MIR_GENE"
    regStem = "NEU"
    
    regeluateWords = ["mediated",]
    
    if mirword.i < geneword.i and "mediated" in str(mirword):
        regStem = "DOWN"
        
    if geneword.i < mirword.i and ("mediated" in str(mirword) or any([str(doc[mirword.i-j]) in ["mediated",] for j in [1]])):
        regStem = "NEU"
        regDir = "MIR_GENE"
        
        return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "return mediated"}
        
    if mirword.i<geneword.i and any([str(doc[geneword.i-j]) in ["by", "with"] for j in [1,2,3]]):
        regStem = "NEU"
        regDir = "MIR_GENE"
        
        return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "return m g by"}
        
    if geneword.i < mirword.i:
        regStem = "NEU"
        regDir = "GENE_MIR"
    
    if geneword.i < mirword.i and any([str(doc[mirword.i-j]) in ["by", "with"] for j in [1,2,3]]):
        regStem = "NEU"
        regDir = "MIR_GENE"
        
    if mirword.i < geneword.i and str(doc[mirword.i+1]) == "target":
        regStem = "NEU"
        regDir = "MIR_GENE"
    
    #if mirword.i < geneword.i:
    #    regDir = "MIR_GENE"
    #else:
    #    regDir = "GENE_MIR"
    
    #if isPassive:
    #    if mirword.i < geneword.i:
    #        regDir = "GENE_MIR"
    #    else:
    #        regDir = "MIR_GENE"
        
    return {"regulation_dir": regStem, "interaction_dir": regDir, "passive": isPassive, "reg_detect": "return"}




In [ ]:
def getstem(tkn):
    tknstr = str(tkn).lower()
    for sidx, s in enumerate(stem2class):
        if tknstr.startswith(s):
            
            if tknstr in ["enhancer","inhibitor"]:
                continue

            tokenStemClass = stem2class[s]
            
            if tokenStemClass == "NEU":
                for i in range(1,2):

                    if tkn.i-i<0:
                        continue

                    #tknPre = str(tkn.doc[tkn.i-i])
                    #if tknPre.startswith(("high", "elevat", "increas")):
                    #    tokenStemClass = "POS"

                    #if tknPre.startswith(("low", "decreas")):
                    #    tokenStemClass = "NEG"
                                                
            return tokenStemClass, s, True
        
        elif tknstr.endswith(("-stressed")):
            return "POS", s, True
        
    return "NEU", str(tkn), False

In [ ]:
allStems, allRels, stem2class = loadStems()

print(stem2class["underexpress"])
print("underexpression".startswith("underexpress"))

doct = nlp("expressed , miR-34a , was studied further because sequence analysis suggested a likely interaction with the 3'-untranslated region of bcl2 mRNA")

toks = [x for x in doct]

for tok in toks:
    print(tok, getstem(tok))

countStems(toks, toks[0], toks[-1])

In [ ]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier()

runCheck(-1, "TRAIN")

In [ ]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier()

runCheck(-1, "TEST")

In [ ]:


def testRelClassifier(doc, mirword, geneword, relchecker, verbose):
    
    if verbose:
        print()
        print()
        print()
        print("Rel Classifier")
    
        print(doc.text_with_ws)
    
    foundStems = set()
    sdpRes = relchecker._get_sdp_path(doc, mirword.i, geneword.i)
    if verbose:
        for x in sdpRes:
            print(x)
        
        
    sdpTokens = [x[0] for x in sdpRes]
    #print(sdpRes)
    
    sdpVerbRels = set()

    for sidx, x in enumerate(sdpRes):
        if x[1] == "VERB":
            for s in allStems:
                if s in str(x[0]):
                    sdpVerbRels.add((s, sidx, stem2class[s]))
                    if verbose:
                        print("sdpVerbRels", x[0], s, sidx, stem2class[s])




            
    docRels = set()
    for tkn in doc:

        if tkn in sdpTokens:
            continue

        for s in allStems:
            if str(tkn).lower().startswith(s):
                docRels.add((s, sidx, stem2class[s]))
                if verbose:    
                    print("docRels", tkn, s, sidx, stem2class[s])
            
            
    mirAncests = [x for x in mirword.ancestors]
    geneAncests = [x for x in geneword.ancestors if not x in mirAncests]
    
    if len(mirAncests) > 1:
        useMirAncests = mirAncests[0:1]
    else:
        useMirAncests = mirAncests
        
    if len(geneAncests) > 1:
        useGeneAncests = geneAncests[0:1]
    else:
        useGeneAncests = geneAncests
        
    useGeneAncests = [x for x in useGeneAncests if not x in useMirAncests]
    
    if verbose:
        print("mirc", mirAncests)
        print("genec", geneAncests)
    
    invCor = False
    regdir = "NEU"
    intdir = "MIR_GENE"
    isPassive = relchecker.checkPassive(doc, mirword, geneword, verbose=False)
    if verbose:
        print("passive?", isPassive)
    
    ancestHandled = False
    #if len(mirAncests) > 0 and len(geneAncests) > 0:
    if True:
        mirRels = []
        geneRels = []
        
        mirRels = getRelsForTokens(useMirAncests, "ancestRels mir", verbose)     
        neuMirs = [x for x in mirRels if x[2] in ["NEU"]]
        #if len(neuMirs) > 0 or len(mirRels) == 0:
        #    mirRels = getRelsForTokens(mirAncests, "ancestRels all mir", verbose)
            
        geneRels = getRelsForTokens(useGeneAncests, "ancestRels gene", verbose)       
        neuGenes = [x for x in geneRels if x[2] in ["NEU"]]
                
        if False and ((len(neuGenes) == len(geneRels)) or len(geneRels) == 0):
            geneRels = getRelsForTokens([x for x in geneAncests if not x in useMirAncests], "ancestRels all gene", verbose)
            neuGenes = [x for x in geneRels if x[2] in ["NEU"]]
            
            if (len(neuGenes) == len(geneRels)) or len(geneRels) == 0:
                
                if mirword.i < geneword.i:
                    tks = [x for x in doc if geneword.i -8 < x.i < geneword.i if not x in useMirAncests]
                else:
                    tks = [x for x in doc if geneword.i < x.i < geneword.i+5 if not x in useMirAncests]
                
                geneRels = getRelsForTokens(tks, "ancestRels tks gene", verbose)
                
                                        
        posRegs = [x for x in mirRels+geneRels if x[2] in ["POS"]]
        negRegs = [x for x in mirRels+geneRels if x[2] in ["NEG"]]
        
        mirPos = len([x for x in mirRels if x[2] in ["POS"]])
        mirNeu = [x for x in mirRels if x[2] in ["NEU"]]
        mirNeg = len([x for x in mirRels if x[2] in ["NEG"]])
        
        genePos = len([x for x in geneRels if x[2] in ["POS"]])
        geneNeu = [x for x in geneRels if x[2] in ["NEU"]]
        geneNeg = len([x for x in geneRels if x[2] in ["NEG"]])
    
        dirReg = (mirPos - geneNeg) + (genePos-mirNeg)
        
        invCor = False
        
        if verbose:
            print(mirword, geneword)
            print("ancestRel counts", mirPos, mirNeg, genePos, geneNeg)
                          
            
        nregdir, ninvcor = evalCounts(dirReg, mirPos, mirNeg, genePos, geneNeg)
        if verbose:
            print("evalcount")
            print(nregdir, ninvcor)
        
        if nregdir != None:
            regdir = nregdir
            invcor = ninvcor
            
            ancestHandled = True
            
            if verbose:
                print("accepted by evalcount")
            
        else:
            lookBehind = 8

                                
            if len(mirNeu) > 0 or len(geneNeu) > 0:
                if verbose:
                    print("ancestRel neu part")

                if mirword.i < geneword.i:
                    for x in mirNeu:
                        if x[0].startswith(("correlat", "affect")):

                            for j in range(1,lookBehind):
                                if str(doc[x[4].i-j]).startswith(("posit", "high")):
                                    mirPos += 1
                                    
                                elif str(doc[x[4].i-j]).startswith(("negat", "low")):
                                    mirNeg += 1
                else:
                    for x in geneNeu:
                        if x[0].startswith(("correlat")):
                            
                            for j in range(1,lookBehind):
                                if str(doc[x[4].i-j]).startswith(("posit", "high")):
                                    genePos += 1
                                elif str(doc[x[4].i-j]).startswith(("negat", "low")):
                                    geneNeg += 1    
            
                                
            if not ancestHandled:    
                
                                
                    
                if verbose:
                    print("ancestRel counts stem check", mirPos, mirNeg, genePos, geneNeg)
                    
                nregdir, ninvcor = evalCounts(dirReg, mirPos, mirNeg, genePos, geneNeg)
                if nregdir != None:
                    regdir = nregdir
                    invcor = ninvcor
                    ancestHandled = True
                    
                    if verbose:
                        print("accept after before stem check")
                    
                if not ancestHandled:
                    for j in range(1,lookBehind):
                        
                        if mirword.i-j < 0:
                            continue
                        
                        jtk = doc[mirword.i-j]
                        if verbose:
                            print("word check mir", j, jtk)
                        pos, neg = scoreWord(jtk)
                        
                        mirPos += pos
                        mirNeg += neg
                            
                    for j in range(1,lookBehind):
                        
                        if geneword.i-j < 0:
                            continue
                        
                        jtk = doc[geneword.i-j]
                        if verbose:
                            print("word check gene", j, jtk)
                            
                        pos, neg = scoreWord(jtk)
                        
                        genePos += pos
                        geneNeg += neg
                            
                            
                if verbose:
                    print("ancestRel counts word check", mirPos, mirNeg, genePos, geneNeg)
                
                nregdir, ninvcor = evalCounts(dirReg, mirPos, mirNeg, genePos, geneNeg)
                if nregdir != None:
                    regdir = nregdir
                    invcor = ninvcor
                    ancestHandled = True
                    
                    if verbose:
                        print("accept after before word check")
                    
           

                
                
        if verbose:
            print("ancestRels", dirReg, invCor)
            
    if ancestHandled:
        
        if isPassive:
            intdir = "GENE_MIR"
            
    
    if not ancestHandled:
        
        if verbose:
            print("ancestHandled part")
        
        if geneword.i < mirword.i:
            lword = geneword
            rword = mirword
            useTks = [x for x in doc if geneword.i < x.i < mirword.i]
        else:
            lword = mirword
            rword = geneword
            useTks = [x for x in doc if mirword.i < x.i < geneword.i]
            
        if verbose:
            print(useTks)
        
        betRels = getRelsForTokens(useTks, "ancestRels useTks", verbose)
        
        if verbose:
            print(betRels)
        
        betPos = len([x for x in betRels if x[2] in ["POS"]])
        betNeg = len([x for x in betRels if x[2] in ["NEG"]])
        
        if betPos + betNeg == 1:
            ancestHandled = True
            
            if verbose:
                print("ancest bet handled", mirword.i, geneword.i, isPassive)
            
            if geneword.i < mirword.i:
                
                if isPassive:
                    intdir = "MIR_GENE"
                    regdir = "DOWN" if betNeg > 0 else "UP"
                    
                else:
                    intdir = "GENE_MIR"
                    regdir = "UP" if betPos > 0 else "DOWN"
                    
            else:
                
                if isPassive:
                    intdir = "GENE_MIR"
                    regdir = "UP" if betPos > 0 else "DOWN"
                    
                else:
                    intdir = "MIR_GENE"
                    regdir = "UP" if betPos > 0 else "DOWN"
                    
        else:
            
            isMediated = False
            
            for x in [lword] + useTks[0:2]:
                for y in ["mediated", "-mediated"]:
                    
                    if y in str(x):
                        isMediated = True
                        break
                if isMediated:
                    break
                                    
            if isMediated:
                
                ancestHandled = True
                
                if verbose:
                    print("ancest mediated handled")
                
                if geneword.i < mirword.i:
                    intdir = "GENE_MIR"
                else:
                    intdir = "MIR_GENE"
                    
                if betPos > betNeg:
                    regdir = "UP"
                elif betNeg > betPos:
                    regdir = "DOWN"
                        
            
            
            

    #classCounter = Counter()
    #if len(classCounter) > 0:
    #    regdir = classCounter.most_common(1)[0][0]
    #else:
    #    regdir = "NEU"
        
    #if invCor:
    #    regdir = "DOWN"       
              
    
    if not ancestHandled:
        
        if verbose:
            print("in final resolution")
        
        
        if geneword.i < mirword.i:
        
            textBetween = " ".join([str(x) for x in doc if geneword.i < x.i < mirword.i])
            textBetween = ' '.join(textBetween.split())
            
            if verbose:
                print(textBetween)
            

            if len(textBetween) < 10 and any([x in textBetween for x in ["by"]]):
                sdpRels = set()
                for sidx,x  in enumerate(sdpRes):
                    for s in allStems:
                        if str(x[0]).lower().startswith(s):
                            sdpRels.add((s, sidx, stem2class[s]))
                            if verbose:
                                print("sdpRels", x[0], s, sidx, stem2class[s])

                posRegs = [x for x in sdpRels if x[2] in ["POS"]]
                negRegs = [x for x in sdpRels if x[2] in ["NEG"]]

                dirReg = len(posRegs)-len(negRegs)

                if dirReg > 0:
                    regdir  = "UP"
                elif dirReg < 0:
                    regdir = "DOWN"
                else:
                    regdir = "NEU"
                
                intdir = "MIR_GENE"

            elif any([x in textBetween for x in ["target of", "target for", "targeting"]]):
                intdir = "MIR_GENE"
                regdir = "NEU"

            else:                
                intdir = "GENE_MIR"
    
        elif mirword.i < geneword.i:
            textBetween = " ".join([str(x) for x in doc if geneword.i-5 < x.i < geneword.i])

            if any([x in textBetween for x in ["by"]]):
                intdir = "GENE_MIR"
                regdir = "UP"
        
    
    regstems = foundStems
    
    #displacy.render(doc, style="dep")
    
    if verbose:
        print((intdir, regdir, "NA", isPassive))
        
    return {"regulation_dir": regdir, "interaction_dir": intdir, "regulation_keys": regstems}